# T2-1. 타이타닉(Titanic) Simple Baseline
---
생존여부 예측모델 만들기
학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 accuracy 평가지표에 따라 채점)


In [656]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./Titanic/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 11), (179, 11), (712, 2), (179, 2))

In [657]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
90,91,3,"Christmann, Mr. Emil",male,29.0,0,0,343276,8.0500,NaN,S
103,104,3,"Johansson, Mr. Gustaf Joel",male,33.0,0,0,7540,8.6542,NaN,S
577,578,1,"Silvey, Mrs. William Baird (Alice Munger)",female,39.0,1,0,13507,55.9000,E44,S
215,216,1,"Newell, Miss. Madeleine",female,31.0,1,0,35273,113.2750,D36,C
191,192,2,"Carbines, Mr. William",male,19.0,0,0,28424,13.0000,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
621,622,1,"Kimball, Mr. Edwin Nelson Jr",male,42.0,1,0,11753,52.5542,D19,S
128,129,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,F E69,C
57,58,3,"Novel, Mr. Mansouer",male,28.5,0,0,2697,7.2292,NaN,C
341,342,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0000,C23 C25 C27,S


In [658]:
y_train

,PassengerId,Survived
90,91,0
103,104,0
577,578,1
215,216,1
191,192,0
...,...,...
621,622,1
128,129,1
57,58,0
341,342,1


In [659]:
X_train.info(),y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 90 to 116
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Pclass       712 non-null    int64  
 2   Name         712 non-null    object 
 3   Sex          712 non-null    object 
 4   Age          575 non-null    float64
 5   SibSp        712 non-null    int64  
 6   Parch        712 non-null    int64  
 7   Ticket       712 non-null    object 
 8   Fare         712 non-null    float64
 9   Cabin        170 non-null    object 
 10  Embarked     711 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 66.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 90 to 116
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  712 non-null    int64
 1   Survived     712 non-null    int64
dtypes: int64(2)
mem

(None, None)

In [660]:
X_train.corr(),y_train['Survived'].value_counts()

C:\Users\USER\AppData\Local\Temp\ipykernel_10432\3530638818.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train.corr(),y_train['Survived'].value_counts()


(             PassengerId    Pclass       Age     SibSp     Parch      Fare
 PassengerId     1.000000 -0.037308  0.022813 -0.048605 -0.035620 -0.002682
 Pclass         -0.037308  1.000000 -0.367589  0.093593  0.011343 -0.563574
 Age             0.022813 -0.367589  1.000000 -0.320248 -0.222482  0.090794
 SibSp          -0.048605  0.093593 -0.320248  1.000000  0.430720  0.160399
 Parch          -0.035620  0.011343 -0.222482  0.430720  1.000000  0.231739
 Fare           -0.002682 -0.563574  0.090794  0.160399  0.231739  1.000000,
 0    441
 1    271
 Name: Survived, dtype: int64)

In [661]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 210 to 45
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  179 non-null    int64  
 1   Pclass       179 non-null    int64  
 2   Name         179 non-null    object 
 3   Sex          179 non-null    object 
 4   Age          139 non-null    float64
 5   SibSp        179 non-null    int64  
 6   Parch        179 non-null    int64  
 7   Ticket       179 non-null    object 
 8   Fare         179 non-null    float64
 9   Cabin        34 non-null     object 
 10  Embarked     178 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 16.8+ KB


In [662]:
X_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
210,211,3,"Ali, Mr. Ahmed",male,24.0,0,0,SOTON/O.Q. 3101311,7.0500,NaN,S
876,877,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534,9.8458,NaN,S
666,667,2,"Butler, Mr. Reginald Fenton",male,25.0,0,0,234686,13.0000,NaN,S
819,820,3,"Skoog, Master. Karl Thorsten",male,10.0,3,2,347088,27.9000,NaN,S
736,737,3,"Ford, Mrs. Edward (Margaret Ann Watson)",female,48.0,1,3,W./C. 6608,34.3750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
494,495,3,"Stanley, Mr. Edward Roland",male,21.0,0,0,A/4 45380,8.0500,NaN,S
871,872,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
530,531,2,"Quick, Miss. Phyllis May",female,2.0,1,1,26360,26.0000,NaN,S
157,158,3,"Corn, Mr. Harry",male,30.0,0,0,SOTON/OQ 392090,8.0500,NaN,S


In [663]:
y_test

,PassengerId,Survived
210,211,0
876,877,0
666,667,0
819,820,0
736,737,0
...,...,...
494,495,0
871,872,1
530,531,1
157,158,0


# 전처리
---

In [664]:
X_train['Embarked'] = X_train['Embarked'].fillna(X_train['Embarked'].mode)
X_test['Embarked'] = X_test['Embarked'].fillna(X_test['Embarked'].mode)

In [665]:
X_train.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            137
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          542
Embarked         0
dtype: int64

In [666]:
onehot_train = pd.get_dummies(X_train[['Sex','Embarked']])
onehot_test = pd.get_dummies(X_test[['Sex','Embarked']])

In [667]:
X_train = X_train.drop(['Cabin','Name','Sex','Embarked','Ticket','Age'],axis=1)
X_test = X_test.drop(['Cabin','Name','Sex','Embarked','Ticket','Age'],axis=1)
X_train = pd.concat([X_train,onehot_train],axis=1)
X_test = pd.concat([X_test,onehot_test],axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [668]:
X_train

array([[0.1011236 , 1.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.11573034, 1.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.64831461, 0.        , 0.125     , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.06404494, 1.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.38314607, 0.        , 0.375     , ..., 0.        , 0.        ,
        1.        ],
       [0.13033708, 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ]])

# 모델 평가

In [669]:
y_train=y_train['Survived']
y_test=y_test['Survived']

In [670]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,y_train)
pred_y = model.predict(X_test)
model.score(X_train,y_train)

1.0

In [671]:
pred_y,y_test

(array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 0, 0], dtype=int64),
 210    0
 876    0
 666    0
 819    0
 736    0
       ..
 494    0
 871    1
 530    1
 157    0
 45     0
 Name: Survived, Length: 179, dtype: int64)

In [672]:
model.score(X_test,y_test)

0.7653631284916201